In [ ]:
import numpy as np
import processingIO as pio
import matplotlib.pyplot as plt
import pandas as pd

from pathlib import Path

from forceClass import Forces
from forceBinsClass import ForceBins

from scipy import stats
from scipy.optimize import curve_fit
from scipy.stats.distributions import  t

from mpl_toolkits.axes_grid1.inset_locator import (inset_axes, InsetPosition,
                                                  mark_inset)

In [ ]:
def power_law(xData, a, b, c):
        return -a*xData**b + c

def fit_data(xData, yData):

    popt, pcov = curve_fit(power_law, xData, yData, p0=[0.5, 0.5, 0.5])

    residuals = yData - power_law(xData, *popt)

    ss_res = np.sum(residuals**2)
    ss_total = np.sum((yData-np.mean(yData))**2)

    r_squared = 1 - (ss_res/ss_total)

    return popt, r_squared

def fit_data_poly(xData, yData, rank):
        tmp_poly = np.polyfit(xData, yData, rank)
        return np.poly1d(tmp_poly)

In [ ]:
# Initial Parameters

# location of the data
data_location = r'/location/of/data'
# location of figures
figure_location = r'../../figures'

# Flags to handle data correctly
force_flag = 1
forceBin_flag = 1

# locations locations if mutliple locations
force_locations = np.array(['force_total', 'forces_top', 'forces_bottom'])
#force_locations = np.array(['forces'])

# cycle information
total_cycles = 10
averaged_cycles = 10

# density of the fluid
density = 1025
# viscosity of the fluid
kinVisc = 1.31e-6

# dynamic parameters for fish
# coefficients for motion
c1, c2 = (-0.825, 1.625)
# wavenumber
lam = 0.95
k = 2 * np.pi / lam
# tail amplitude
amp = 0.1
# omega's related to strouhal
omega = np.array([0, 3.141592654, 9.424777961, 15.70796327, 18.84955592, 21.99114858, 26.70353756, 31.41592654, 34.55751919, 37.69911184])
# fluid Velocity
U = 1.0
L = 1.0
# strouhal numbers tested
st = (2*amp)*(omega/(2*np.pi))/U

# Reynolds numbers tested
Re = np.array([3e2, 4e3])


In [ ]:
force_paths = pio.get_files(data_location, 'force.dat')
force_paths.sort()
forceBins_paths = pio.get_files(data_location, 'forceBin.dat')
forceBins_paths.sort()


if force_flag == 1:
    # get the individual cases
    cases = []
    force_case = force_paths[0].parts[-7]
    cases.append(force_case)
    for force_file in force_paths:
        if force_file.parts[-7] != force_case:
            cases.append(force_file.parts[-7])
            force_case = force_file.parts[-7]

    # make a nice dict of all the file names based on parent case
    force_caseDict = dict()
    for case in cases:
        force_caseDict[case] = {}
        for location in force_locations:
            force_caseDict[case][location] = []
            for force_file in force_paths:
                if force_file.parts[-7] == case and force_file.parts[-3] == location:
                    force_caseDict[case][location].append(force_file)

if forceBin_flag == 1:
    # get the individual cases
    cases = []
    forceBin_case = forceBins_paths[0].parts[-7]
    cases.append(forceBin_case)
    for forceBins_file in forceBins_paths:
        if forceBins_file.parts[-7] != forceBin_case:
            cases.append(forceBins_file.parts[-7])
            forceBin_case = forceBins_file.parts[-7]

    # make a nice dict of all the file names based on parent case
    forceBin_caseDict = dict()
    for case in cases:
        forceBin_caseDict[case] = {}
        for location in force_locations:
            forceBin_caseDict[case][location] = []
            for forceBins_file in forceBins_paths:
                if forceBins_file.parts[-7] == case and forceBins_file.parts[-3] == location:
                    forceBin_caseDict[case][location].append(forceBins_file)

In [ ]:
force_paths_total = []

if force_flag == 1:
    for case in force_caseDict:
        for path in force_caseDict[case][force_locations[0]]:
            force_paths_total.append(path)
    forces = [Forces(force_path, averaged_cycles, total_cycles, True, True, 'blackman', 51) for force_path in force_paths_total]

In [ ]:
plt.rcParams.update({'font.size':18})

fig1, ax1 = plt.subplots(figsize=(12,10))

for force in forces[0:9]:
    ax1.plot(force.filteredForces['time']/force.filteredForces['time'][-1]*averaged_cycles, force.filteredForces['total']['x']/(density*U**2*L**2), label=force.specific_case)

ax1.set_xlabel('t/T')
ax1.set_ylabel('Force coefficient')
ax1.legend(ncol=4, bbox_to_anchor=(1.05, 1.14))
ax1.set_ylim([-5, 5])
#fig1.savefig(Path(figure_location).joinpath('AvgPower_St.png'))

In [ ]:
forceBin_paths_top = []
forceBin_paths_bottom = []

if forceBin_flag == 1:
    for case in forceBin_caseDict:
        for path in forceBin_caseDict[case][force_locations[1]]:
            forceBin_paths_top.append(path)
        for path in forceBin_caseDict[case][force_locations[2]]:
            forceBin_paths_bottom.append(path)    

forceBins_top = [ForceBins(forceBin_path, averaged_cycles, total_cycles, True, True, 'blackman', 51) for forceBin_path in forceBin_paths_top]
forceBins_bottom = [ForceBins(forceBin_path, averaged_cycles, total_cycles, True, True, 'blackman', 51) for forceBin_path in forceBin_paths_bottom]

In [ ]:
for forceBin in forceBins_top:
    if forceBin.specific_case == 'st0_00':
        forceBin.calcPowerCarangiform(density, np.array([amp, c1, c2]), omega[0], k)
    elif forceBin.specific_case == 'st0_10':
        forceBin.calcPowerCarangiform(density, np.array([amp, c1, c2]), omega[1], k)
    elif forceBin.specific_case == 'st0_30':
        forceBin.calcPowerCarangiform(density, np.array([amp, c1, c2]), omega[2], k)
    elif forceBin.specific_case == 'st0_50':
        forceBin.calcPowerCarangiform(density, np.array([amp, c1, c2]), omega[3], k)
    elif forceBin.specific_case == 'st0_60':
        forceBin.calcPowerCarangiform(density, np.array([amp, c1, c2]), omega[4], k)
    elif forceBin.specific_case == 'st0_70':
        forceBin.calcPowerCarangiform(density, np.array([amp, c1, c2]), omega[5], k)
    elif forceBin.specific_case == 'st0_85':
        forceBin.calcPowerCarangiform(density, np.array([amp, c1, c2]), omega[6], k)
    elif forceBin.specific_case == 'st1_00':
        forceBin.calcPowerCarangiform(density, np.array([amp, c1, c2]), omega[7], k)
    elif forceBin.specific_case == 'st1_10':
        forceBin.calcPowerCarangiform(density, np.array([amp, c1, c2]), omega[8], k)
    elif forceBin.specific_case == 'st1_20':
        forceBin.calcPowerCarangiform(density, np.array([amp, c1, c2]), omega[9], k)

for forceBin in forceBins_bottom:
    if forceBin.specific_case == 'st0_00':
        forceBin.calcPowerCarangiform(density, np.array([amp, c1, c2]), omega[0], k)
    elif forceBin.specific_case == 'st0_10':
        forceBin.calcPowerCarangiform(density, np.array([amp, c1, c2]), omega[1], k)
    elif forceBin.specific_case == 'st0_30':
        forceBin.calcPowerCarangiform(density, np.array([amp, c1, c2]), omega[2], k)
    elif forceBin.specific_case == 'st0_50':
        forceBin.calcPowerCarangiform(density, np.array([amp, c1, c2]), omega[3], k)
    elif forceBin.specific_case == 'st0_60':
        forceBin.calcPowerCarangiform(density, np.array([amp, c1, c2]), omega[4], k)
    elif forceBin.specific_case == 'st0_70':
        forceBin.calcPowerCarangiform(density, np.array([amp, c1, c2]), omega[5], k)
    elif forceBin.specific_case == 'st0_85':
        forceBin.calcPowerCarangiform(density, np.array([amp, c1, c2]), omega[6], k)
    elif forceBin.specific_case == 'st1_00':
        forceBin.calcPowerCarangiform(density, np.array([amp, c1, c2]), omega[7], k)
    elif forceBin.specific_case == 'st1_10':
        forceBin.calcPowerCarangiform(density, np.array([amp, c1, c2]), omega[8], k)
    elif forceBin.specific_case == 'st1_20':
        forceBin.calcPowerCarangiform(density, np.array([amp, c1, c2]), omega[9], k)

In [ ]:
base_coeff = -1.0

if forceBin_flag == 1:
    average_power_dict = dict()
    for case in forceBin_caseDict:
        average_power_dict[case] = {force_locations[1]:[], force_locations[2]:[], 'combined':[]}
        for forceBin in forceBins_top:
            if forceBin.parent_case == case:
                average_power_dict[case][force_locations[1]].append(forceBin.average_power)
        for forceBin in forceBins_bottom:
            if forceBin.parent_case == case:
                average_power_dict[case][force_locations[2]].append(forceBin.average_power)

        average_power_dict[case]['combined'] = (np.array(average_power_dict[case][force_locations[1]]) + 
                                                np.array(average_power_dict[case][force_locations[2]]))

if force_flag == 1:
    average_force_dict = dict()
    for case in force_caseDict:
        average_force_dict[case] = []
        for force in forces:
            if force.parent_case == case:
                T_t = (0.5 * (-force.averageFilteredForces['pressure']['x'] + np.abs(force.averageFilteredForces['pressure']['x'])) + 
                            0.5 * (force.averageFilteredForces['viscous']['x'] + np.abs(force.averageFilteredForces['viscous']['x'])))
                        
                D_t = (0.5 * (force.averageFilteredForces['pressure']['x'] - np.abs(force.averageFilteredForces['pressure']['x'])) + 
                            0.5 * (force.averageFilteredForces['viscous']['x'] - np.abs(force.averageFilteredForces['viscous']['x'])))

                F_t = np.array(T_t - D_t) / density
                
                
                average_force_dict[case].append(F_t)

In [ ]:
average_power_fit = {}
average_force_fit = {}
zero_strouhal = {}

for case in force_caseDict:
    average_power_fit[case] = {}
    average_force_fit[case] = {}
    
    if forceBin_flag == 1:
        case_power_fit = fit_data_poly(st, np.array(average_power_dict[case]['combined']), 2)
        average_power_fit[case]['fit'] = case_power_fit
        average_power_fit[case]['roots'] = case_power_fit.r

    if force_flag == 1:
        case_force_fit = fit_data_poly(st, np.array(average_force_dict[case])- 1, 2)
        case_power_zero = fit_data_poly(np.array(average_force_dict[case]) - 1, st, 2)
        average_force_fit[case]['fit']= case_force_fit
        average_force_fit[case]['roots'] = case_force_fit.r
        
    zero_strouhal[case] = case_power_zero(0)

In [ ]:
fig2, (ax1, ax2) = plt.subplots(1, 2, figsize=(22,10))
for case in average_force_dict:
    ax1.plot(st, np.array(average_force_dict[case]) - 1, 'o', label = case)
    ax2.plot(st, np.array(average_force_fit[case]['fit'](st)), label = case + ' fit')

borazjani_Re3e02_st = np.array([0, 0.1, 0.2, 0.3, 0.6, 0.8, 1.0, 1.1, 1.2]) 
borazjani_Re3e02_ft = np.array([-1.001372998, -1.042618084, -1.010170745, -0.968128851, -0.739461362, -0.482379863, -0.148402277, 0.050635452, 0.268878719])

borazjani_Re4e03_st = np.array([0, 0.1, 0.2, 0.3, 0.5, 0.6, 0.61, 0.70]) 
borazjani_Re4e03_ft = np.array([-1.001356569, -1.25085607, -1.080659508, -0.913666608, -0.419505956, -0.018645778, 0.016635569, 0.452686734])

ax1.plot(borazjani_Re3e02_st, borazjani_Re3e02_ft, 'o-', color='C0', label='bor_Re3e02')
ax1.plot(borazjani_Re4e03_st, borazjani_Re4e03_ft, 'o-', color='orange', label='bor_Re4e03')

ax2.plot(borazjani_Re3e02_st, borazjani_Re3e02_ft, 'o--', color='C0', label='bor_Re3e02')
ax2.plot(borazjani_Re4e03_st, borazjani_Re4e03_ft, 'o--', color='orange', label='bor_Re4e03')

ax1.plot([0, 1.2], [-1.0, -1.0], 'k--')
ax1.plot([0.0, 1.2], [0, 0], 'k--')

ax2.plot([0, 1.2], [-1.0, -1.0], 'k--')
ax2.plot([0.0, 1.2], [0, 0], 'k--')

ax1.set_xlabel('Strouhal')
ax2.set_xlabel('Strouhal')
ax1.set_ylabel('Mean Force Coefficient')
ax1.legend(ncol=2, bbox_to_anchor=(0.58, 1.0))
ax2.legend(ncol=2, bbox_to_anchor=(0.585, 1.0))
#ax1.set_ylim([-1.25, 0.25])
#ax2.set_ylim([-1.25, 0.25])
#fig2.savefig(Path(figure_location).joinpath('AvgPower_St.png'))

In [ ]:
fig3, (ax1, ax2) = plt.subplots(1, 2, figsize=(22,10))
for case in average_force_dict:
    ax1.plot(st, average_power_dict[case]['combined'], 'o--', label = case)
    ax2.plot(st, average_power_fit[case]['fit'](st), label = case + ' fit')


ax1.set_xlabel('Strouhal')
ax2.set_xlabel('Strouhal')
ax1.set_ylabel('Average Power [W]')
ax1.legend(ncol=3, bbox_to_anchor=(1.02, 1.18))
ax2.legend(ncol=3, bbox_to_anchor=(1.05, 1.18))
#fig2.savefig(Path(figure_location).joinpath('AvgPower_St.png'))

In [ ]:
fig4, ax4 = plt.subplots(figsize=(12,10))
for case in average_power_dict:
    ax4.plot(zero_strouhal[case], average_power_fit[case]['fit'](zero_strouhal[case]), 'o', label = case)

ax4.set_xlabel('Strouhal')
ax4.set_ylabel('Average Power [W]')
ax4.legend(ncol=4, bbox_to_anchor=(1.05, 1.14))
#fig4.savefig(Path(figure_location).joinpath('AvgPower_St.png'))

In [ ]:
fig4, ax4 = plt.subplots(figsize=(12,10))
for i, case in enumerate(forceBin_caseDict):
    ax4.plot(Re[i], average_power_fit[case]['fit'](zero_strouhal[case]), 'o')

ax4.set_xscale('log')
ax4.set_xlabel('Re')
ax4.set_ylabel('Average Power [W]')
#fig4.savefig(Path(figure_location).joinpath('AvgPower_Re.png'))

In [ ]:
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
borazjani_zero_st = np.array([1.08, 0.6])

Re_plot = np.linspace(0.1, 1e8, 200)
St_plot_low = 2.3*Re_plot**(-1/4)

fig5, ax5 = plt.subplots(figsize=(12,10))
for i, case in enumerate(forceBin_caseDict):
    ax5.plot(Re[i], zero_strouhal[case], 'o', label=case)
    ax5.plot(Re[i], borazjani_zero_st[i], 'd', label='Borazjani ' + str(Re[i]))

ax5.plot(Re_plot, St_plot_low, 'k')
ax5.plot([0.1, 1e8], [0.3, 0.3], 'k--')

ax5.set_xscale('log')
ax5.set_yscale('log')
ax5.set_xlabel('Re')
ax5.set_ylabel('St')
ax5.set_ylim([1e-1, 1e1])
ax5.xaxis.set_minor_locator(AutoMinorLocator())
ax5.tick_params(which='major', length=7)
ax5.tick_params(which='minor', length=4)
ax5.legend()
#fig5.savefig(Path(figure_location).joinpath('Re_St.png'))

In [ ]:
Sw_plot = np.linspace(1, 1e9, 200)
Re_low = 0.04*Sw_plot**(4/3)
Re_high = 0.43*Sw_plot 

fig6, ax6 = plt.subplots(figsize=(12,10))
for i, case in enumerate(forceBin_caseDict):
    Sw = zero_strouhal[case] * Re[i]
    Sw_borazjani = borazjani_zero_st[i] * Re[i]
    ax6.plot(Sw, Re[i], 'o', label=case)
    ax6.plot(Sw_borazjani, Re[i], 'd', label='Borazjani ' + str(Re[i]))

ax6.plot(Sw_plot, Re_low, 'k')
ax6.plot(Sw_plot, Re_high, 'k--')

ax6.set_yscale('log')
ax6.set_xscale('log')
ax6.set_xlabel('Sw')
ax6.set_ylabel('Re')
ax6.set_xlim([1e2, 1e5])
ax6.set_ylim([1e1, 1e4])
ax6.legend()
#fig6.savefig(Path(figure_location).joinpath('Sw_Re.png'))